# «Смарт» или «Ультра»

Изучим данные, предоставленные сервисом для проекта.

## шаг. Открыть файл с данными и изучить его.

In [1]:
#импортируем необходимые библиотеки
import pandas as pd
import numpy as np
import random
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.dummy import DummyClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import RandomForestRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.linear_model import LinearRegression

In [2]:
# чтение файла с данными в случае локальной работы и сохранение в df
try:
    df = pd.read_csv(r'E:\DS\users_behavior.csv')
# чтение файла с данными в случае работы online и сохранение в df
except:
    df = pd.read_csv('/datasets/users_behavior.csv') 

df.head()

,calls,minutes,messages,mb_used,is_ultra
0,40.0,311.90,83.0,19915.42,0
1,85.0,516.75,56.0,22696.96,0
2,77.0,467.66,86.0,21060.45,0
3,106.0,745.53,81.0,8437.39,1
4,66.0,418.74,1.0,14502.75,0


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3214 entries, 0 to 3213
Data columns (total 5 columns):
calls       3214 non-null float64
minutes     3214 non-null float64
messages    3214 non-null float64
mb_used     3214 non-null float64
is_ultra    3214 non-null int64
dtypes: float64(4), int64(1)
memory usage: 125.7 KB


В датасете 5 столбцов, пропусков в данных нет, стилистических и синтаксических нарушений нет.<br/>
Обратимся к описанию данных:<br/>
<b>сalls </b> — количество звонков;<br/>
<b>minutes</b> — суммарная длительность звонков в минутах;<br/>
<b>messages</b> — количество sms-сообщений;<br/>
<b>mb_used</b> — израсходованный интернет-трафик в Мб;<br/>
<b>is_ultra</b> — каким тарифом пользовался в течение месяца («Ультра» — 1, «Смарт» — 0). 

**Вывод.**<br/>
Поскольку данные не нуждаются в предобработке, переходим к дальнейшему исследованию. 

## шаг. Разделить исходные данные на обучающую, валидационную и тестовую выборки.

Спрятанной тестовой выборки нет. Значит, данные нужно разбить на три части: обучающую, валидационную и тестовую. Размеры тестового и валидационного наборов обычно равны. Исходные данные разбивают в соотношении 3:1:1.

In [4]:
#признаки и цели для исходного датасета
features = df.drop(['is_ultra'], axis=1)
target = df['is_ultra']

#разделим исходные данные на три выборки
train, validate, test = np.split(df.sample(frac=1, random_state=12345),  [int(.6*len(df)), int(.8*len(df))])

#признаки и цель для обучающей выборки
features_train = train.drop(['is_ultra'], axis=1)
target_train = train['is_ultra']

#признаки и цель для валидационной выборки
features_validate = validate.drop(['is_ultra'], axis=1)
target_validate = validate['is_ultra']

#признаки и цель для тестовой выборки
features_test = test.drop(['is_ultra'], axis=1)
target_test = test['is_ultra']

    
#посмотрим на размер новых даатасетов
def info_df(features_train, target_train, features_validate, target_validate, features_test, target_test): #на вход функция принимает все датасеты сразу
    df_all = [features_train, target_train, features_validate, target_validate, features_test, target_test]
    for i in df_all:
        print(i.shape)
                    
print(info_df(features_train, target_train, features_validate, target_validate, features_test, target_test))

(1928, 4)
(1928,)
(643, 4)
(643,)
(643, 4)
(643,)
None


### Исследовать качество разных моделей, меняя гиперпараметры. Кратко напишите выводы исследования.

Проверять на тестовой выборке пока не нужно, сначала найдём наилучшую модель.

In [5]:
#Дерево решений
ff = []

for depth in range(1, 6):
    model = DecisionTreeClassifier(random_state=12345, max_depth=depth) 
    
    model.fit(features_train, target_train) #обучаем на тренировочной выборке,
    
    predictions_validate = model.predict(features_validate) #а предсказание - на валидационной
           
    ff.append({'max_depth': depth, 'accuracy': accuracy_score(target_validate, predictions_validate)})   
        
ff = pd.DataFrame(ff)
print(max(ff['accuracy']))
print(ff.sort_values('accuracy', ascending=False))

0.7744945567651633
   max_depth  accuracy
3          4  0.774495
4          5  0.772939
2          3  0.766719
1          2  0.751166
0          1  0.721617


In [6]:
#Случайный лес
best_model = None
best_result = 0
for est in range(1, 11):
    model = RandomForestClassifier(random_state=12345, n_estimators=est) #обучаем модель с заданным количеством деревьев
    model.fit(features_train, target_train) #обучаем модель на тренировочной выборке
    result = model.score(features_validate, target_validate) #посчитаем качество модели на валидационной выборке
    if result > best_result:
        best_model = model #сохраняем наилучшую модель
        best_result = result #сохраняем наилучшее значение метрики accuracy на валидационных данных

print("Accuracy модели случайный лес на валидационной выборке:", best_result)

Accuracy модели случайный лес на валидационной выборке: 0.7807153965785381


In [7]:
#Случайный лес
best_model = None
best_result = 0
for est in range(1, 11):
    for n_samples in range(1, 20):
        for depth in range(1, 15):
            model = RandomForestClassifier(  #обучаем с заданными параметрами:
                random_state=12345, 
                max_depth = depth, #максимальная глубина деревьев
                n_estimators=est,  #число «деревьев» в «случайном лесу»
                n_jobs = -1, #распараллеливание,используются все ядра, имеющиеся на машине 
                max_features = 'auto', #число признаков, по которым ищется разбиение
                min_samples_leaf = n_samples)  #минимальное число объектов в листьях
            model.fit(features_train, target_train) #обучаем модель на тренировочной выборке
            result = model.score(features_validate, target_validate) #посчитаем кач-во модели на валидационной выборке
            if result > best_result:
                best_model = model #сохраняем наилучшую модель
                best_result = result #сохраняем наилучшее значение метрики accuracy на валидационных данных

print("Accuracy модели случайный лес на валидационной выборке:", best_result)

Accuracy модели случайный лес на валидационной выборке: 0.8040435458786936


In [8]:
#Логистическая регрессия
model = LogisticRegression(random_state=12345) #инициализируем модель логистической регрессии с параметром random_state=12345
model.fit(features_train, target_train) #обучим модель на тренировочной выборке
result = model.score(features_validate, target_validate) #получим метрику качества модели на валидационной выборке
 
print("Accuracy модели логистической регрессии на валидационной выборке:", result)

Accuracy модели логистической регрессии на валидационной выборке: 0.6765163297045101


/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


Итак наилучшей моделью определён "случайный лес".

### Проверить качество модели на тестовой выборке.

Чем выше качество модели, тем лучше. Но её внедрение должно быть оправданно. Очевидные границы: accuracy не может быть меньше нуля (все ответы неправильные) и больше единицы (все ответы правильные). 

In [9]:
model = RandomForestClassifier(random_state=12345, n_estimators=100) #обучаем модель с заданным количеством деревьев
model.fit(features_train, target_train) #обучаем модель на тренировочной выборке
target_predict = model.predict(features_test) #предсказание на тесте
pred = model.score(features_test, target_test) #посчитаем качество модели на тестовой выборке
print("Accuracy модели случайный лес на тестовой выборке:", pred)

Accuracy модели случайный лес на тестовой выборке: 0.7993779160186625


Отличается ли accuracy на валидационной и тестовой выборках необходимо узнать.

In [10]:
#отличается ли accuracy на валидационной и тестовой выборках
model = DecisionTreeClassifier(random_state=12345)
model.fit(features_train, target_train)
 
predictions_validate = model.predict(features_validate)
accuracy = accuracy_score(target_validate, predictions_validate) 
 
predictions_test = model.predict(features_test)
accuracy_test = accuracy_score(target_test, predictions_test) 
 
print("Accuracy")
print("Валидационная выборка:", accuracy)
print("Тестовая выборка:", accuracy_test)

Accuracy
Валидационная выборка: 0.71850699844479
Тестовая выборка: 0.7402799377916018


**Вывод**

Итак наилучшей моделью определён "случайный лес". Его Accuracy на валидационной выборке: 0.7807153965785381. Accuracy на обучающей и тестовой выборках немного разнятся.

## шаг. Дополнительное задание: проверить модели на вменяемость.

Чтобы оценить адекватность модели в задачах классификации, нужно сравнить её со случайной.

In [11]:
#создадим столбец в исходном датасете с набором случайно распределенных значений (т.е. с вероятностью 0.5 получаем либо 1, либо 0)
a = np.array([0, 1])

df['random_column'] = np.random.choice(a, 3214, replace=True) 

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3214 entries, 0 to 3213
Data columns (total 6 columns):
calls            3214 non-null float64
minutes          3214 non-null float64
messages         3214 non-null float64
mb_used          3214 non-null float64
is_ultra         3214 non-null int64
random_column    3214 non-null int64
dtypes: float64(4), int64(2)
memory usage: 150.8 KB


In [12]:
#признаки и цели для нового датасета
features_new = df.drop(['is_ultra', 'random_column'], axis=1)
target_new = df['random_column']

#разделим исходные данные на две выборки
train_new, validate_new = train_test_split(df, test_size=0.25, random_state=12345)

#признаки и цель для обучающей выборки
features_train_new = train_new.drop(['is_ultra', 'random_column'], axis=1)
target_train_new = train_new['random_column']

#признаки и цель для валидационной выборки
features_validate_new = validate_new.drop(['is_ultra', 'random_column'], axis=1)
target_validate_new = validate_new['random_column']

    
#посмотрим на размер новых даатасетов
def info_df(features_train_new, target_train_new, features_validate_new, target_validate_new): #на вход функция принимает все датасеты сразу
    df_all = [features_train_new, target_train_new, features_validate_new, target_validate_new]
    for i in df_all:
        print(i.shape)
                    
print(info_df(features_train_new, target_train_new, features_validate_new, target_validate_new))

(2410, 4)
(2410,)
(804, 4)
(804,)
None


Лучшей моделью был определён Случайный лес. Обучим её и проверим, а далее сравним полученное accuracy с accuracy предыдущим.

In [13]:
#Случайный лес
best_model = None
best_result = 0
for est in range(1, 11):
    model = RandomForestClassifier(random_state=12345, n_estimators=est) #обучаем модель с заданным количеством деревьев
    model.fit(features_train_new, target_train_new) #обучаем модель на тренировочной выборке
    result = model.score(features_validate_new, target_validate_new) #посчитаем качество модели на валидационной выборке
    if result > best_result:
        best_model = model #сохраняем наилучшую модель
        best_result = result #сохраняем наилучшее значение метрики accuracy на валидационных данных
 
print("Accuracy модели случайный лес на валидационной выборке:", best_result)

Accuracy модели случайный лес на валидационной выборке: 0.5398009950248757


In [14]:
#отличается ли accuracy на обучающей и тестовой выборках
model = DecisionTreeClassifier(random_state=12345)
model.fit(features_new, target_new)
 
predictions_train_new = model.predict(features_train_new)
accuracy = accuracy_score(target_train_new, predictions_train_new) 
 
predictions_test = model.predict(features_test)
accuracy_test = accuracy_score(target_test, predictions_test) 
 
print("Accuracy")
print("Обучающая выборка:", accuracy)
print("Тестовая выборка:", accuracy_test)

Accuracy
Обучающая выборка: 1.0
Тестовая выборка: 0.48833592534992226


Accuracy тестовой выборки намного ниже чем Accuracy обучающей. Значит случайная модель хорошо справилась с обучающим набором данных, но с тестовым у неё возникли сложности. Получается, что выбранная наилучшая модель Случайный лес вполне вменяема и показала лучшие результаты по сравнению со случайной моделью.

Самый частотный класс - 0.

In [19]:
strategies=['most_frequent']

predictions_results = []
for strategy in strategies:
    random_model = DummyClassifier(strategy=strategy, random_state = 1235)
    random_model.fit(features, target)
    results = random_model.score(features, target)
    predictions_results.append({strategy: results})
    
pd.DataFrame(predictions_results)

,most_frequent
0,0.693528


In [16]:
#посчитаем точность большего класса
df['is_ultra'].value_counts() / df.shape[0]

0    0.693528
1    0.306472
Name: is_ultra, dtype: float64

### шаг. Написать общий вывод.

<body>
    Итак наилучшей моделью определён "случайный лес". Его Accuracy на валидационной выборке составило 0.78. После проверки выбранной модели Accuracy на обучающей и тестовой выборках оказались одинаковые. Предсказания и ответы совпали.<br/>
    При проверки модели на вменяемость, мы сравнили её со случайной, и получили результаты:<br/>
    Accuracy тестовой выборки намного ниже чем Accuracy обучающей. Значсит случайная модель хорошо справилась с обучающим набором данных, но с тестовым у неё возникли сложности.<br/>
    Получается, что выбранная наилучшая модель Случайный лес вполне вменяема и показала лучшие результаты по сравнению со случайной моделью.
    </body>